# Setup


In [1]:
# python >= 3.5 is required
import sys
assert sys.version_info >= (3,5)
# scikit learn >=0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

# commmon imports
import numpy as np
import os

# to maake this notebook outpuut stable accross runs
np.random.seed(42)

# set up plot 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes',labelsize=14)
mpl.rc('xtick',labelsize=12)
mpl.rc('ytick',labelsize=12)

# where to save the figure
ROOT = '.'
CHAPTER_ID = 'svm'
IMAGES_PATH = os.path.join(ROOT,'images',CHAPTER_ID)
os.makedirs(IMAGES_PATH,exist_ok=True)

def save_fig(fig_id,tight_layout=True, fig_extension='png',resolution=300):
    path = os.path.join(IMAGES_PATH,fig_id + '.' +fig_extension)
    print('saving images figure',fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path,format=fig_extension,dpi=resolution)


In [7]:
from sklearn.datasets import load_iris
iris = load_iris()

X = iris['data']
y = iris['target']

In [10]:
import numpy as np
X1D = np.linspace(-4,4,9).reshape(1,-1)
X2D = np.c_[X1D,X1D**2]# make the data kuadratic

y = np.array([0, 0, 1, 1, 1, 1, 1, 0, 0])
y 

array([0, 0, 1, 1, 1, 1, 1, 0, 0])